# 2D meshing: xy cross-section

You can supply the argument `type="xy"` and a `z`-value, to mesh arbitrary `Component` planar cross-sections.

In [ ]:
import gdsfactory as gf
from gdsfactory.simulation.gmsh.mesh import create_physical_mesh
import meshio
import pyvista as pv

waveguide = gf.components.straight_pin(length=10, taper=None)
waveguide

In [ ]:
from gdsfactory.tech import LayerMap, LayerStack, LayerLevel, get_layer_stack_generic

filtered_layerstack = LayerStack(
    layers={
        k: get_layer_stack_generic().layers[k]
        for k in (
            "slab90",
            "core",
            "via_contact",
        )
    }
)

In [ ]:
filename = "mesh"


def mesh_with_physicals(mesh, filename):
    mesh_from_file = meshio.read(f"{filename}.msh")
    triangle_mesh = create_physical_mesh(mesh_from_file, "triangle", prune_z=True)
    meshio.write(f"{filename}.xdmf", triangle_mesh)
    mesh = pv.read(f"{filename}.xdmf")
    return mesh

At `z=0.09` um, according to the layer stack above we should see polygons from all three layers:

In [ ]:
filename = "mesh"
mesh = waveguide.to_gmsh(
    type="xy", z=0.09, layer_stack=filtered_layerstack, filename=f"{filename}.msh"
)
mesh = mesh_with_physicals(mesh, filename)

In [ ]:
plotter = pv.Plotter()
plotter.add_mesh(mesh, style="wireframe", show_scalar_bar=False)
plotter.show_grid()
plotter.camera_position = "xy"
plotter.show()

At `z=0`, you can see only the core and slab:

In [ ]:
mesh = waveguide.to_gmsh(
    type="xy", z=0.0, layer_stack=filtered_layerstack, filename=f"{filename}.msh"
)
mesh = mesh_with_physicals(mesh, filename)

In [ ]:
plotter = pv.Plotter()
plotter.add_mesh(mesh, style="wireframe", show_scalar_bar=False)
plotter.show_grid()
plotter.camera_position = "xy"
plotter.show()

At `z=1.0`, you can only see the vias appear:

In [ ]:
mesh = waveguide.to_gmsh(
    type="xy", z=1.0, layer_stack=filtered_layerstack, filename=f"{filename}.msh"
)
mesh = mesh_with_physicals(mesh, filename)

In [ ]:
plotter = pv.Plotter()
plotter.add_mesh(mesh, style="wireframe", show_scalar_bar=False)
plotter.show_grid()
plotter.camera_position = "xy"
plotter.show()

## Controlling meshing domain

You can use functions that return other components to modify the simulation domain, for instance `gdsfactory.geometry.trim`:

In [ ]:
waveguide_trimmed = gf.Component()
waveguide_trimmed.add_ref(
    gf.geometry.trim(
        component=waveguide,
        domain=[[3, -4], [3, 4], [5, 4], [5, -4]],
    )
)

waveguide_trimmed

In [ ]:
mesh = waveguide_trimmed.to_gmsh(
    type="xy", z=0.09, layer_stack=filtered_layerstack, filename=f"{filename}.msh"
)
mesh = mesh_with_physicals(mesh, filename)

In [ ]:
plotter = pv.Plotter()
plotter.add_mesh(mesh, style="wireframe", show_scalar_bar=False)
plotter.show_grid()
plotter.camera_position = "xy"
plotter.show()